# Fase 2: Validación del Sistema de Recuperación (Retrieval)

## 1. Introducción
Tras la ingesta de datos realizada en la fase anterior, entramos en la etapa de recuperación de información. Según la arquitectura RAG, el sistema debe ser capaz de identificar documentos relevantes ("chunks") en base a una consulta del usuario.

En este notebook verificaremos la eficacia del índice `noticias_tfg` utilizando algoritmos de búsqueda léxica.

## 2. Objetivos
1.  **Verificación de Indexación:** Confirmar que el volumen de datos disponible es correcto tras el filtrado ETL.
2.  **Prueba de Motor de Búsqueda:** Ejecutar consultas de prueba para evaluar la latencia y la precisión.
3.  **Análisis de Relevancia (Scoring):** Examinar cómo el algoritmo BM25 de Elasticsearch puntúa los documentos recuperados y ver si los documentos recuperados tienen o no coherencia con la consulta realizada

In [ ]:
import os
import json
from elasticsearch import Elasticsearch

# --- CONFIGURACIÓN ---
# Apuntamos al puerto 9250 y forzamos IPv4
ES_HOST = "http://127.0.0.1:9250"
INDEX_NAME = "noticias_tfg"

# Desactivamos proxies para evitar conflictos en la red universitaria
os.environ["NO_PROXY"] = "localhost,127.0.0.1"

print(f"Conectando al sistema de recuperación en {ES_HOST}...")
es = Elasticsearch(ES_HOST, request_timeout=30) # request_timeout para evitar bloqueos largos en caso de problemas de conexión

# Verificación de salud del índice
if es.ping():
    if es.indices.exists(index=INDEX_NAME):
        count = es.count(index=INDEX_NAME)['count']
        print(f"CONEXIÓN EXITOSA.")
        print(f"Estado del Índice: ACTIVO")
        print(f"Documentos disponibles para RAG: {count} noticias")
    else:
        print(f"ALERTA: Conexión ok, pero el índice '{INDEX_NAME}' no existe. ¿Ejecutaste el Notebook 01?")
else:
    print("ERROR: No se puede conectar a Elasticsearch. Revisa la terminal.")

Conectando al sistema de recuperación en http://127.0.0.1:9250...
CONEXIÓN EXITOSA.
Estado del Índice: ACTIVO
Documentos disponibles para RAG: 490956 noticias


In [4]:
def retrieve_context(query, k=5):
    """
    Ejecuta una búsqueda de texto completo (Full-Text Search) sobre el índice de noticias.
    Simula el componente 'Retriever' de una arquitectura RAG.
    
    Args:
        query (str): La pregunta o tema a buscar.
        k (int): Número de documentos a recuperar (Top-K).
    """
    print(f"\nCONSULTA: '{query}'")
    print("=" * 80)
    
    # Definición de la Query (Sintaxis DSL de Elasticsearch)
    # Usamos 'multi_match' para buscar en título y cuerpo a la vez
    search_payload = {
        "size": k,
        "query": {
            "multi_match": {
                "query": query,
                "fields": ["title^3", "body"], # Damos 3 veces más importancia (boost) a palabras en el título
                "fuzziness": "AUTO",           # fuzziness para tolerancia a errores tipográficos leves
                "operator": "or"               # Permite que no todas las palabras tengan que coincidir, aumentando la flexibilidad de la consulta
            }
        },
        "_source": ["title", "date", "url", "source"] # Solo traemos campos útiles para esta prueba, no el cuerpo entero para no ensuciar
    }
    
    try:
        response = es.search(index=INDEX_NAME, body=search_payload) # Ejecutamos la búsqueda
        hits = response['hits']['hits'] # Extraemos los resultados (hits) de la respuesta
        
        if not hits:
            print("No se encontraron resultados relevantes.")
            return

        # Visualización de resultados
        for i, hit in enumerate(hits):
            score = hit['_score'] # Relevancia del resultado según Elasticsearch (BM25)
            data = hit['_source'] 
            clean_date = data.get('date', 'N/A')[:10] if data.get('date') else "Sin fecha"
            
            print(f"Resultado {i+1} | Relevancia (Score): {score:.4f}")
            print(f" Fecha:  {clean_date}")
            print(f" Título: {data.get('title', 'Sin título')}")
            print(f" Fuente: {data.get('source', 'Desconocida')}")
            print("-" * 80)
            
    except Exception as e:
        print(f"Error durante la búsqueda: {e}")

print("Función 'retrieve_context' cargada correctamente.")

Función 'retrieve_context' cargada correctamente.


In [8]:
# CASO 1: Búsqueda Factual Específica (Economía)
retrieve_context("guerra comercial Trump aranceles China")

# CASO 2: Búsqueda de Tema Reciente (Tecnología/Sociedad)
retrieve_context("inteligencia artificial regulación")

# CASO 3: Prueba de Robustez (Erratas intencionadas)
# Escribimos "Mbape" mal (sin tilde y con una p) para ver si 'fuzziness' funciona
retrieve_context("fichaje Kylian Mbape Real Madrid")

# CASO 4: Búsqueda de Tema General (Salud)
retrieve_context("vacunas COVID-19 llevan un chip")


CONSULTA: 'guerra comercial Trump aranceles China'
Resultado 1 | Relevancia (Score): 78.9104
 Fecha:  2025-04-14
 Título: Trump amenaza con más aranceles a móviles en su guerra comercial con China
 Fuente: La Razón
--------------------------------------------------------------------------------
Resultado 2 | Relevancia (Score): 78.9104
 Fecha:  2025-04-10
 Título: Guerra comercial: Aranceles de Trump a China ascienden a 145%
 Fuente: El Universal
--------------------------------------------------------------------------------
Resultado 3 | Relevancia (Score): 78.9104
 Fecha:  2024-09-21
 Título: Aranceles a China: Apoyan en EU guerra comercial de Trump
 Fuente: Excélsior
--------------------------------------------------------------------------------
Resultado 4 | Relevancia (Score): 74.9799
 Fecha:  2025-04-16
 Título: Trump grava a China con aranceles de hasta el 245% por «sus represalias» en la guerra comercial
 Fuente: ABC.es
-------------------------------------------------------